In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/NovKaggle/test.csv.zip' '/content/test.csv.zip'

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/NovKaggle/train.csv.zip' '/content/train.csv.zip'
!cp '/content/drive/MyDrive/NovKaggle/sample_submission.csv.zip' '/content/sample_submission.csv.zip'


In [ ]:
!unzip -q train.csv.zip -d /content/
!unzip -q test.csv.zip -d /content/
!unzip -q sample_submission.csv.zip -d /content/

In [ ]:
#@title Texto de título padrão
#Data Exploration
import pandas as pd
train_df =  pd.read_csv('/content/train.csv',index_col='id')

In [ ]:
print("Number of columns (included column target):",len(train_df.columns))
print("Number of rows:",len(train_df))

Number of columns (included column target): 101
Number of rows: 600000


In [ ]:
train_df['target']

id
0         0
1         0
2         0
3         0
4         1
         ..
599995    1
599996    0
599997    0
599998    1
599999    1
Name: target, Length: 600000, dtype: int64

In [ ]:
train_df.head()

<bound method NDFrame.head of               f0        f1         f2  ...       f98       f99  target
id                                     ...                            
0       0.106643  3.594370   132.8040  ...  0.085974  0.240496       0
1       0.125021  1.673360    76.5336  ... -0.020877  0.024719       0
2       0.036330  1.497470   233.5460  ...  0.124464  0.148209       0
3      -0.014077  0.246000   779.9670  ...  0.040017  0.044873       0
4      -0.003259  3.715420   156.1280  ... -0.012819  0.072798       1
...          ...       ...        ...  ...       ...       ...     ...
599995  0.043008  1.640550  1375.8900  ...  0.049038  0.065804       1
599996  0.137048  4.826860   -99.6478  ...  0.162313  0.085662       0
599997  1.316220  0.502062    66.2360  ...  0.084864  0.052635       0
599998  0.511918  3.827470   777.8720  ...  0.061334  0.004179       1
599999  0.179347  3.457390   160.5980  ...  0.049075  0.044436       1

[600000 rows x 101 columns]>

In [ ]:
train_df.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
count,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,...,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000
mean,0.306508,2.497590,306.644536,2.647901,0.177850,2.556832,2.699650,2.571593,2.538273,0.134370,2.579987,2.759273,0.128958,2.477244,2.662859,2.631040,0.362479,2.483075,2.471319,0.108097,0.093783,0.062157,2.559781,0.097061,0.141055,2.580093,2.459101,0.057945,0.216789,2.471344,0.125638,0.107798,0.170434,0.054776,2.463409,55.698848,1.769510,2.595079,2.410926,0.593497,...,0.144262,0.124303,0.340198,0.086385,2.513444,2.577005,2.522315,0.134467,0.189364,2.494925,2.632673,0.129834,0.050814,2.551266,0.174057,0.231822,2.420256,0.101771,0.349875,2.550175,0.902983,2.598029,0.162773,0.501982,2.587270,2.653461,0.265197,0.335756,0.056866,0.071252,2.444471,0.155260,0.059407,0.144932,0.106419,2.547853,2.590159,0.158881,0.123048,0.506010
std,0.522450,1.554018,551.743893,1.544529,0.417488,1.562527,1.564000,1.549361,1.532988,0.421892,1.604389,1.545217,0.210861,1.517694,1.481966,1.531067,1.346352,1.577456,1.541571,0.395313,0.308938,0.054796,1.556496,0.333590,0.342377,1.609573,1.567891,0.114129,0.339320,1.524330,0.296563,0.412291,0.440615,0.109661,1.566572,130.132892,1.785712,1.562506,1.572078,0.951892,...,0.377962,0.414040,0.623011,0.216007,1.516342,1.563820,1.532317,0.398998,0.744550,1.575232,1.541872,0.308313,0.049142,1.513612,0.641809,1.206276,1.557891,0.245734,0.596830,1.576794,1.335611,1.542775,0.295831,2.364291,1.564173,1.546943,0.562230,0.993598,0.128607,0.112654,1.542509,0.548397,0.119426,0.462015,0.209128,1.558427,1.525091,0.436190,0.264896,0.499964
min,-3.797450,-1.223960,-1842.530000,-1.368560,-3.206210,-1.169770,-1.059310,-1.281970,-1.242020,-2.577840,-1.309730,-1.451570,-0.678145,-1.295160,-1.213890,-1.450540,-18.169600,-1.387720,-1.221020,-5.334110,-2.858550,-0.151519,-1.201070,-2.721660,-2.645250,-1.380430,-1.147260,-2.281050,-4.596700,-1.225650,-2.025710,-3.273000,-1.954520,-3.318420,-1.517720,-397.004000,-2.888940,-1.210370,-1.181050,-2.611230,...,-1.326570,-2.553080,-5.153130,-3.498950,-1.237310,-1.260920,-1.059910,-2.987210,-11.650200,-1.377760,-1.236730,-4.609500,-0.659289,-1.134010,-8.940060,-18.456000,-1.164970,-1.425230,-4.397590,-1.142840,-8.519100,-1.135490,-1.516430,-9.791940,-1.307630,-1.383700,-2.544860,-15.470500,-4.421610,-3.676990,-1.217700,-9.761770,-4.666240,-3.101500,-1.276540,-1.584740,-1.254730,-3.993500,-2.783380,0.000000
25%,0.026222,1.186238,43.573400,1.442028,0.019709,1.261038,1.385820,1.333848,1.292163,0.019563,1.205920,1.520750,0.029119,1.278118,1.504670,1.399320,-0.046389,1.132497,1.218018,0.017039,0.018545,0.027432,1.287208,0.011614,0.017647,1.238715,1.156460,-0.018820,0.057385,1.25

In [ ]:
#missing values?
train_df.isnull().any().any()

False

split the data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data_features = train_df.drop(['target'], axis=1)
data_targets = train_df['target']

X_train, X_val, y_train, y_val = train_test_split(data_features, data_targets, test_size=0.2, random_state=3)


In [ ]:
#preprocess the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [ ]:
type(X_train_scaled)

numpy.ndarray

PyTorch - Neural Network

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np


In [ ]:
class BinaryDataset(Dataset):
  def __init__(self, input_array, target_array):
    self.input_array = input_array
    self.target_array
    
  def len(self):
    return len(self.input_array)

  def getitem(self, idx):
    x=torch.tensor(self.input_array[idx], dtype=torch.float) #needs to be a tensor, type of the tensor
    y=torch.tensor(self.target_array[idx], dtype=torch.float)
    return x,y 

class LinearNetwork(nn.Module):
  def __init__ (self):  #self is from pytorch library
    super(LinearNetwork, self).__init__()#init = starting/initial function
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(100,512),
        nn.ReLU(),
        nn.Linear(512,1028),
        nn.ReLU(),
        nn.Dropout(p=0.4), #during training with probability of p, it will zero
                            #some neurons, to prevent overfitting, because the network will always have blindspots that change every duration, so the network has to adapt
        nn.Linear(1028, 512),
        nn.Dropout(p=0.4),
        nn.ReLu(),        
        nn.Linear(512, 256),
        nn.Dropout(p=0.4),
        nn.Relu(),
        nn.Linear(256, 1),
    )
  def foward(self,x):
    return self.linear_relu_stack(x)


In [ ]:
train_dataset = BinaryDataset(X_train_scaled, np.array(y_train))
val_dataset = BinaryDataset(X_val_scaled, np.array(y_val))

train